<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/verbs_cosine_w2v_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz


--2020-02-25 06:02:01--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz’

news_mystem_skipgra 100%[===================>] 524.47M  28.5MB/s    in 19s     

2020-02-25 06:02:20 (27.4 MB/s) - ‘news_mystem_skipgram_1000_20_2015.bin.gz’ saved [549952184/549952184]



In [1]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.8MB/s 
     |████████████████████████████████| 7.1MB 7.3MB/s 
     |████████████████████████████████| 378kB 46.5MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=851656 sha256=9fdc7c01e815c9b2d004d1e5e85d135461e661faf4979148189b0ee81876c44b
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [3]:
#!rm collection.json.1 collection.json
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

--2020-02-25 06:02:21--  https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 606414 (592K) [text/plain]
Saving to: ‘collection.json’

collection.json     100%[===================>] 592.20K  --.-KB/s    in 0.06s   

2020-02-25 06:02:21 (9.88 MB/s) - ‘collection.json’ saved [606414/606414]



In [4]:
!wget http://vectors.nlpl.eu/repository/20/187.zip

--2020-02-25 06:02:23--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  22.8MB/s    in 1m 57s  

2020-02-25 06:04:20 (21.9 MB/s) - ‘187.zip’ saved [2692389554/2692389554]



In [5]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [6]:
!ls

187.zip		 model.model.vectors_ngrams.npy		   README
collection.json  model.model.vectors.npy		   sample_data
meta.json	 model.model.vectors_vocab.npy
model.model	 news_mystem_skipgram_1000_20_2015.bin.gz


In [0]:
import gensim
from collections import Counter
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [8]:
skipgram_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
fasttext_model = gensim.models.KeyedVectors.load("model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# rv_model.vocab
# rv_model.get_vector('помочь_V')

In [0]:
# rv_model.vocab

In [0]:
collection_handler = open("collection.json", "r", encoding="utf-8")
collection = json.load(collection_handler, encoding="utf-8")

In [13]:

def make_group(polarity_type_list):
  """
  На вход подаются пары из разных типов отношений(p_type) в списках(как в переменных all_type_dict и pos_type_dict)
  для каждой пары цикл ищет не более ста глаголов, у которых есть типы отношений, совпадающие с нашей парой, 
  которая подается на вход. 
  """
  group_dict = Counter()
  for p_type in polarity_type_list:
    print(p_type)
    count = 0
    for key in collection.keys():
      if count < 100:
        try:
          #print(collection[key]['frames']['polarity'])
          if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] in collection[key]['frames']['polarity'] :
          # эту строчку можно менять, если нам нужно искать отношения, где только ['a0', 'a1', 'pos', 1.0]:
          # if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] not in collection[key]['frames']['polarity'] : 
          # проблема! в лексиконе всего 20 отношений [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]] c разными коэффициентами, и 51 таких, 
          # где ['a0', 'a1', 'pos', 1.0], а ['a1', 'a0', 'pos', 1.0] отсутствует
            for ver in collection[key]['title']:
              group_dict[ver] = p_type
              count += 1
            for var in collection[key]['variants']:
              if len(var.split(' ')) > 1 and var[-2:] == 'ть':
                group_dict[var] = p_type
                count += 1

              
        except Exception as e:
          pass
  return group_dict

all_type_dict = [[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]], \
                 [['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]], \
                  [['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]], \
                  [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]],
                 ]
pos_type_dict = [[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]], \
                 [['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]], \
            
                 ]
neg_type_dict = [
                  [['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]], \
                  [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]],
                 ]
all_dict = make_group(all_type_dict)
pos_dict = make_group(pos_type_dict)
neg_dict = make_group(neg_type_dict)
#len(groups_total_dict)
print(len(all_dict))
print(len(pos_dict))
print(len(neg_dict))


[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]
[['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]]
[['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]]
[['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]
[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]
[['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]]
[['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]]
[['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]
169
60
109


In [14]:
all_dict.most_common(5)

[('помочь', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('поощрить', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('похвалить', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('приветствовать', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('радовать', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]])]

In [15]:
pos_dict.most_common(5)

[('помочь', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('поощрить', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('похвалить', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('приветствовать', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]),
 ('радовать', [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]])]

In [16]:
neg_dict.most_common(5)

[('отказать', [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]),
 ('ответить отрицательно',
  [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]),
 ('грозить', [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]),
 ('нести угрозу', [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]),
 ('подозревать', [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]])]

## **Для начала сформируем вектор сидового множества с позитивными глаголами.**

In [0]:
seed_one_dict = ['одобрять', 'хвалить', 'поощрять']
def make_seed_embedding(seed_dict, model):
  '''
  Принимает на вход список, на выходе дает средний вектор всех слов из этого списка
  '''
  summ = 0
  if model == skipgram_model:
    for verb in seed_dict:
      summ = summ + model[verb + '_V']
    vector = summ/len(seed_dict)
  else:
    for verb in seed_dict:
      summ = summ + model.get_vector(verb)
    vector = summ/len(seed_dict)

  return vector
seed_one_skipgram = make_seed_embedding(seed_one_dict, skipgram_model)
seed_one_fasttext = make_seed_embedding(seed_one_dict, fasttext_model)

In [22]:
#polar_verbs = [verb for verb in groups_total_dict.keys()]
all_relations_pol = [pol for pol in all_dict.keys()]
pos_relations_pol = [pol for pol in pos_dict.keys()]
neg_relations_pol = [pol for pol in neg_dict.keys()]
print(len(all_relations_pol))
print(len(pos_relations_pol))
print(len(neg_relations_pol))

169
60
109


## **В некоторых случаях попадаются не только глаголы, но и какие-то фразы, содержащие глагол. Далее оставим только глагол.**

In [23]:
def sort_verbs(my_dict):
  '''
  Принимает на вход словарь слов и словосочетаний, если видит, что там словосочетание, 
  оставляет только глагол. Возвращает отфильтрованный список.
  '''
  polar_verbs_list = []
  for phrase in my_dict.keys():
    if len(phrase.split(' ')) > 1:
      for verb in phrase.split(' '):
        if 'VERB' in morph.parse(verb)[0].tag or 'INFN' in morph.parse(verb)[0].tag:
          polar_verbs_list.append(verb)
    else:
      polar_verbs_list.append(phrase)
  print(polar_verbs_list[:10])
  return polar_verbs_list

all_relations_list = sort_verbs(all_dict)
pos_relations_list = sort_verbs(pos_dict)
neg_relations_list = sort_verbs(neg_dict)

print(len(all_dict))
print(len(pos_dict))
print(len(neg_dict))

print(len(all_relations_list))
print(len(pos_relations_list))
print(len(neg_relations_list))


['помочь', 'поощрить', 'похвалить', 'приветствовать', 'радовать', 'дарить', 'доставить', 'доставлять', 'подарить', 'спасти']
['помочь', 'поощрить', 'похвалить', 'приветствовать', 'радовать', 'дарить', 'доставить', 'доставлять', 'подарить', 'спасти']
['повредить', 'испортить', 'обеспокоить', 'потрясти', 'доминировать', 'выследить', 'настичь', 'догнать', 'ворваться', 'ловить']
169
60
109
169
60
109


## **Сделаем датафреймы для всех глаголов вообще, и отдельно для негативных и позитивных.**

In [0]:
df_all = pd.DataFrame({'verbs':all_relations_list, 'phrase':all_relations_pol})
df_pos = pd.DataFrame({'verbs':pos_relations_list, 'phrase':pos_relations_pol})
df_neg = pd.DataFrame({'verbs':neg_relations_list, 'phrase':neg_relations_pol})

## **И положим туда значение косинусной дистанции между вектором сидового множества, и каждым из глаголов. Некоторых глаголов нет в моделях, это отобразится в таблице тоже.**

In [62]:
def add_sim_list(model, my_verbs_list, seed_vector):
  '''
  Принимает на вход модель, как назвать результирующую колонку в датафрейме,
  словарь из глаголов, и средний вектор сидового множества.
  записывает колонку в датафрейм, возвращает словарь косинусных расстояний
  '''
  sim_list = []
  if model == skipgram_model:  
    for verb in my_verbs_list:
        try:
          similarity = cosine_distances(np.atleast_2d(model[verb + '_V']), np.atleast_2d(seed_vector))
        except KeyError as e:
          similarity = 'verb is not in model dict'
        sim_list.append(similarity)
  else:
    for verb in my_verbs_list:
      try:
        similarity = cosine_distances(np.atleast_2d(model.get_vector(verb)), np.atleast_2d(seed_vector))
      except (KeyError, AttributeError) as e:
        similarity = 'verb is not in model dict'
      sim_list.append(similarity)
  print(len(sim_list))
  print(len(my_verbs_list))

  #dataframe[df_pos] = sim_list
  return sim_list

#add_sim_list(rv_model, verbs, opinion_embedding)
df_all['w2v_model'] = add_sim_list(skipgram_model, all_relations_list, seed_one_skipgram)
df_all['fasttext_model'] = add_sim_list(fasttext_model, all_relations_list, seed_one_fasttext)

df_pos['w2v_model'] = add_sim_list(skipgram_model, pos_relations_list, seed_one_skipgram)
df_pos['fasttext_model'] = add_sim_list(fasttext_model, pos_relations_list, seed_one_fasttext)

df_neg['w2v_model'] = add_sim_list(skipgram_model, neg_relations_list, seed_one_skipgram)
df_neg['fasttext_model'] = add_sim_list(fasttext_model, neg_relations_list, seed_one_fasttext)

169
169
169
169
60
60
60
60
109
109
109
109


In [63]:
df_all.head(1)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[0.91887474]]


In [64]:
df_pos.head(30)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[0.91887474]]
1,поощрить,поощрить,verb is not in model dict,[[0.17637241]]
2,похвалить,похвалить,[[0.48269153]],[[0.20982283]]
3,приветствовать,приветствовать,[[0.5898497]],[[0.36748838]]
4,радовать,радовать,[[0.72734857]],[[0.38311976]]
5,дарить,дарить радость,[[0.752011]],[[0.48386037]]
6,доставить,доставить радость,verb is not in model dict,[[0.50851893]]
7,доставлять,доставлять радость,[[0.9204415]],[[0.530205]]
8,подарить,подарить радость,[[0.78662884]],[[0.44142973]]
9,спасти,спасти,verb is not in model dict,[[0.6940966]]


In [65]:
df_neg.head(30)

,verbs,phrase,w2v_model,fasttext_model
0,повредить,повредить,verb is not in model dict,[[0.5450003]]
1,испортить,испортить,[[0.78414404]],[[0.41191667]]
2,обеспокоить,обеспокоить,[[0.7456583]],[[0.5044007]]
3,потрясти,потрясти до глубины души,verb is not in model dict,[[0.69239753]]
4,доминировать,доминировать,[[0.8056042]],[[0.52223873]]
5,выследить,выследить,verb is not in model dict,[[0.52639043]]
6,настичь,настичь,verb is not in model dict,[[0.9443023]]
7,догнать,догнать,[[0.87338156]],[[0.5594841]]
8,ворваться,ворваться,[[0.9255186]],[[0.7055975]]
9,ловить,ловить,[[0.79371715]],[[0.50638425]]


## **Попробуем другие сидовые множества.**

In [0]:
seed_two_dict = ['осуждать', 'негодовать']

In [0]:
seed_two_skipgram = make_seed_embedding(seed_two_dict, skipgram_model)
seed_two_fasttext = make_seed_embedding(seed_two_dict, fasttext_model)

In [0]:
df_all2 = pd.DataFrame({'verbs':all_relations_list, 'phrase':all_relations_pol})
df_pos2 = pd.DataFrame({'verbs':pos_relations_list, 'phrase':pos_relations_pol})
df_neg2 = pd.DataFrame({'verbs':neg_relations_list, 'phrase':neg_relations_pol})

In [69]:
df_all2['w2v_model'] = add_sim_list(skipgram_model, all_relations_list, seed_two_skipgram)
df_all2['fasttext_model'] = add_sim_list(fasttext_model, all_relations_list, seed_two_fasttext)

df_pos2['w2v_model'] = add_sim_list(skipgram_model, pos_relations_list, seed_two_skipgram)
df_pos2['fasttext_model'] = add_sim_list(fasttext_model, pos_relations_list, seed_two_fasttext)

df_neg2['w2v_model'] = add_sim_list(skipgram_model, neg_relations_list, seed_two_skipgram)
df_neg2['fasttext_model'] = add_sim_list(fasttext_model, neg_relations_list, seed_two_fasttext)

169
169
169
169
60
60
60
60
109
109
109
109


In [70]:
df_all2.head(1)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[1.0172709]]


In [71]:
df_pos2.head(10)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[1.0172709]]
1,поощрить,поощрить,verb is not in model dict,[[0.4265676]]
2,похвалить,похвалить,[[0.7065598]],[[0.50444555]]
3,приветствовать,приветствовать,[[0.74483865]],[[0.43908948]]
4,радовать,радовать,[[0.8283391]],[[0.50327724]]
5,дарить,дарить радость,[[0.85249525]],[[0.64122814]]
6,доставить,доставить радость,verb is not in model dict,[[0.5756773]]
7,доставлять,доставлять радость,[[0.91314846]],[[0.53472865]]
8,подарить,подарить радость,[[0.80887055]],[[0.599512]]
9,спасти,спасти,verb is not in model dict,[[0.70674324]]


In [72]:
df_neg2.head(10)

,verbs,phrase,w2v_model,fasttext_model
0,повредить,повредить,verb is not in model dict,[[0.53814566]]
1,испортить,испортить,[[0.77675235]],[[0.43742788]]
2,обеспокоить,обеспокоить,[[0.643383]],[[0.4523908]]
3,потрясти,потрясти до глубины души,verb is not in model dict,[[0.6660273]]
4,доминировать,доминировать,[[0.8884222]],[[0.46882892]]
5,выследить,выследить,verb is not in model dict,[[0.48472315]]
6,настичь,настичь,verb is not in model dict,[[0.97252905]]
7,догнать,догнать,[[0.8723642]],[[0.5780843]]
8,ворваться,ворваться,[[0.7986826]],[[0.63658583]]
9,ловить,ловить,[[0.78541136]],[[0.55598485]]


In [0]:
seed_three_dict = ['обожать', 'восхищаться', 'восторгаться', 'нравиться', 'ценить', 'гордиться']

In [0]:
seed_three_skipgram = make_seed_embedding(seed_three_dict, skipgram_model)
seed_three_fasttext = make_seed_embedding(seed_three_dict, fasttext_model)

In [0]:
df_all3 = pd.DataFrame({'verbs':all_relations_list, 'phrase':all_relations_pol})
df_pos3 = pd.DataFrame({'verbs':pos_relations_list, 'phrase':pos_relations_pol})
df_neg3 = pd.DataFrame({'verbs':neg_relations_list, 'phrase':neg_relations_pol})

In [76]:
df_all3['w2v_model'] = add_sim_list(skipgram_model, all_relations_list, seed_three_skipgram)
df_all3['fasttext_model'] = add_sim_list(fasttext_model, all_relations_list, seed_three_fasttext)

df_pos3['w2v_model'] = add_sim_list(skipgram_model, pos_relations_list, seed_three_skipgram)
df_pos3['fasttext_model'] = add_sim_list(fasttext_model, pos_relations_list, seed_three_fasttext)

df_neg3['w2v_model'] = add_sim_list(skipgram_model, neg_relations_list, seed_three_skipgram)
df_neg3['fasttext_model'] = add_sim_list(fasttext_model, neg_relations_list, seed_three_fasttext)

169
169
169
169
60
60
60
60
109
109
109
109


In [77]:
df_all3.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[1.0869569]]
1,поощрить,поощрить,verb is not in model dict,[[0.42445105]]
2,похвалить,похвалить,[[0.5607922]],[[0.34632283]]
3,приветствовать,приветствовать,[[0.6863141]],[[0.41872245]]
4,радовать,радовать,[[0.5692914]],[[0.267159]]


In [78]:
df_pos3.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[1.0869569]]
1,поощрить,поощрить,verb is not in model dict,[[0.42445105]]
2,похвалить,похвалить,[[0.5607922]],[[0.34632283]]
3,приветствовать,приветствовать,[[0.6863141]],[[0.41872245]]
4,радовать,радовать,[[0.5692914]],[[0.267159]]


In [79]:
df_neg3.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,повредить,повредить,verb is not in model dict,[[0.5909805]]
1,испортить,испортить,[[0.7522595]],[[0.43052638]]
2,обеспокоить,обеспокоить,[[0.74948955]],[[0.55537665]]
3,потрясти,потрясти до глубины души,verb is not in model dict,[[0.5817593]]
4,доминировать,доминировать,[[0.780125]],[[0.49368525]]


In [0]:
seed_four_dict = ['хвалить', 'нахваливать', 'превозносить']

In [0]:
seed_four_skipgram = make_seed_embedding(seed_four_dict, skipgram_model)
seed_four_fasttext = make_seed_embedding(seed_four_dict, fasttext_model)

In [0]:
df_all4 = pd.DataFrame({'verbs':all_relations_list, 'phrase':all_relations_pol})
df_pos4 = pd.DataFrame({'verbs':pos_relations_list, 'phrase':pos_relations_pol})
df_neg4 = pd.DataFrame({'verbs':neg_relations_list, 'phrase':neg_relations_pol})

In [83]:
df_all4['w2v_model'] = add_sim_list(skipgram_model, all_relations_list, seed_four_skipgram)
df_all4['fasttext_model'] = add_sim_list(fasttext_model, all_relations_list, seed_four_fasttext)

df_pos4['w2v_model'] = add_sim_list(skipgram_model, pos_relations_list, seed_four_skipgram)
df_pos4['fasttext_model'] = add_sim_list(fasttext_model, pos_relations_list, seed_four_fasttext)

df_neg4['w2v_model'] = add_sim_list(skipgram_model, neg_relations_list, seed_four_skipgram)
df_neg4['fasttext_model'] = add_sim_list(fasttext_model, neg_relations_list, seed_four_fasttext)

169
169
169
169
60
60
60
60
109
109
109
109


In [84]:
df_all4.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[1.0470661]]
1,поощрить,поощрить,verb is not in model dict,[[0.32173353]]
2,похвалить,похвалить,[[0.4625256]],[[0.16315764]]
3,приветствовать,приветствовать,[[0.73000586]],[[0.41382754]]
4,радовать,радовать,[[0.6887789]],[[0.3956275]]


In [85]:
df_pos4.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[1.0470661]]
1,поощрить,поощрить,verb is not in model dict,[[0.32173353]]
2,похвалить,похвалить,[[0.4625256]],[[0.16315764]]
3,приветствовать,приветствовать,[[0.73000586]],[[0.41382754]]
4,радовать,радовать,[[0.6887789]],[[0.3956275]]


In [86]:
df_neg4.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,повредить,повредить,verb is not in model dict,[[0.51155555]]
1,испортить,испортить,[[0.78878605]],[[0.39415395]]
2,обеспокоить,обеспокоить,[[0.8354587]],[[0.6093259]]
3,потрясти,потрясти до глубины души,verb is not in model dict,[[0.60367393]]
4,доминировать,доминировать,[[0.7563846]],[[0.53989583]]


In [0]:
seed_five_dict = ['знать', 'понимать', 'думать']

In [0]:
seed_five_skipgram = make_seed_embedding(seed_five_dict, skipgram_model)
seed_five_fasttext = make_seed_embedding(seed_five_dict, fasttext_model)

In [0]:
df_all5 = pd.DataFrame({'verbs':all_relations_list, 'phrase':all_relations_pol})
df_pos5 = pd.DataFrame({'verbs':pos_relations_list, 'phrase':pos_relations_pol})
df_neg5 = pd.DataFrame({'verbs':neg_relations_list, 'phrase':neg_relations_pol})

In [90]:
df_all5['w2v_model'] = add_sim_list(skipgram_model, all_relations_list, seed_five_skipgram)
df_all5['fasttext_model'] = add_sim_list(fasttext_model, all_relations_list, seed_five_fasttext)

df_pos5['w2v_model'] = add_sim_list(skipgram_model, pos_relations_list, seed_five_skipgram)
df_pos5['fasttext_model'] = add_sim_list(fasttext_model, pos_relations_list, seed_five_fasttext)

df_neg5['w2v_model'] = add_sim_list(skipgram_model, neg_relations_list, seed_five_skipgram)
df_neg5['fasttext_model'] = add_sim_list(fasttext_model, neg_relations_list, seed_five_fasttext)

169
169
169
169
60
60
60
60
109
109
109
109


In [91]:
df_all5.head(1)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[0.77468705]]


In [92]:
df_pos5.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,помочь,помочь,verb is not in model dict,[[0.77468705]]
1,поощрить,поощрить,verb is not in model dict,[[0.5927094]]
2,похвалить,похвалить,[[0.7282261]],[[0.5880505]]
3,приветствовать,приветствовать,[[0.84770906]],[[0.76272225]]
4,радовать,радовать,[[0.7502494]],[[0.56329155]]


In [93]:
df_neg5.head(5)

,verbs,phrase,w2v_model,fasttext_model
0,повредить,повредить,verb is not in model dict,[[0.68238425]]
1,испортить,испортить,[[0.7806171]],[[0.59634995]]
2,обеспокоить,обеспокоить,[[0.81098634]],[[0.48944563]]
3,потрясти,потрясти до глубины души,verb is not in model dict,[[0.7270875]]
4,доминировать,доминировать,[[0.7912472]],[[0.7196263]]


In [0]:
# vect1 = np.atleast_2d([1, 2, 4])
# vect2 = np.atleast_2d([1, 4, 4])
# print(cosine_distances(vect1, vect2))
# vect1

In [0]:
# def get_embedding(verbs_list, model, dim=50):
    
#     # чтобы не доставать одно слово несколько раз
#     # сделаем счетчик, а потом векторы домножим на частоту
#     words = Counter(verbs_list)
#     total = len(verbs_list)
#     vectors = np.zeros((len(words), dim))
    
#     for i,word in enumerate(words):
#         try:
#             v = model[word]
#             vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
#             print(vectors)
#         except (KeyError, ValueError):
#             continue
    
#     if vectors.any():
#         vector = np.average(vectors, axis=0)
#     else:
#         vector = np.zeros((dim))
#     return vector

# opinion_embedding = get_embedding(opinion_dict, rv_model)
# opinion_embedding

In [0]:
# opinion_dict = ['одобрять_V', 'хвалить_V', 'поощрять_V']
# def make_sid_embedding(seed_dict, model):
#   summ = 0
#   try:
#     for verb in seed_dict:
#       #print(model[verb])
#       summ = summ + get_embedding(verb, model)
#       print(summ)
#   except KeyError:
#     pass
#   vector = summ/len(seed_dict)
#   return vector
# opinion_embedding = make_sid_embedding(opinion_dict, rv_model)

In [0]:
# opinion_embedding

In [0]:
def make_similarity_df(base_vect, verbs_for_check_dict, model):
  '''
  Считает косинусное расстояние с каким-либо подающимся на вход сидовым глаголом глаголов из словаря, полученного с помощью 
  функции make_group. Возвращает словарь с ключами 'groups', 'base_verb', 'verb' и 'verbs_similarity', 
  которые потом будут колонками таблицы - датафрейма.
  '''

  df_dict = Counter()
  df_dict['groups'] = []
  df_dict['base_verb'] = []
  df_dict['verb'] = []
  df_dict['verbs_similarity'] = []

  #my_base_verb = base_verb + '_V'
  for verb in verbs_for_check_dict.keys():
    verb_for_check = verb + '_V'
    try:
      verbs_similarity = model.similarity(base_vect, verb_for_check)
      df_dict['groups'].append(str(verbs_for_check_dict[verb]))
      df_dict['base_verb'].append(base_verb)
      df_dict['verb'].append(verb)
      df_dict['verbs_similarity'].append(verbs_similarity)
    except KeyError as e:
      pass
      #print(e)
  return df_dict

verbs_for_df = make_similarity_df(opinion_embedding, groups_total_dict, rv_model)
print('done')

done


In [0]:

frames_df = pd.DataFrame(verbs_for_df, columns = verbs_for_df.keys()) # тут делается датафрейм

In [0]:
frames_df

,groups,base_verb,verb,verbs_similarity


In [0]:

# verbs_for_check_dict = {'снимать': [['a0', 'a1', 'neg', 1.0]]}

# def just_words_similarity(base_verb, verbs_for_check_dict):
#   """
#   Функция тестовая, проверяет, как работает косинусная близость.
#   """
#   group_dict = Counter()
#   my_base_verb = base_verb + '_V'
#   for verb in verbs_for_check_dict.keys():
#     verb_for_check = verb + '_V'
#     similarity = mystem_skipgram_model.similarity(my_base_verb, verb_for_check)
#     group_dict[str(verbs_for_check_dict[verb])] = [base_verb, verb, similarity]
#   return group_dict

# just_words_similarity('радоваться', verbs_for_check_dict)


